In [70]:
from dotenv import dotenv_values
config = dotenv_values(".env")
API_KEY = config["API_KEY"]
API_SECRET_KEY = config["API_SECRET_KEY"]
BEARER_TOKEN = "paste here"
ACCESS_TOKEN = "paste from twitter response"
ACCESS_TOKEN_SECRET = "paste from twitter response"

In [ ]:
from requests_oauthlib import OAuth1Session
import json


consumer_key = API_KEY
consumer_secret = API_SECRET_KEY

# Be sure to add replace the text of the with the text you wish to Tweet. You can also add parameters to post polls, quote Tweets, Tweet with reply settings, and Tweet to Super Followers in addition to other features.
payload = {"text": "Hello world!"}

# Get request token
request_token_url = "https://api.twitter.com/oauth/request_token?oauth_callback=oob&x_auth_access_type=write"
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

try:
    fetch_response = oauth.fetch_request_token(request_token_url)
except ValueError:
    print(
        "There may have been an issue with the consumer_key or consumer_secret you entered."
    )

resource_owner_key = fetch_response.get("oauth_token")
resource_owner_secret = fetch_response.get("oauth_token_secret")
print("Got OAuth token: %s" % resource_owner_key)

# Get authorization
base_authorization_url = "https://api.twitter.com/oauth/authorize"
authorization_url = oauth.authorization_url(base_authorization_url)
print("Please go here and authorize: %s" % authorization_url)
verifier = input("Paste the PIN here: ")

# Get the access token
access_token_url = "https://api.twitter.com/oauth/access_token"
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=resource_owner_key,
    resource_owner_secret=resource_owner_secret,
    verifier=verifier,
)
oauth_tokens = oauth.fetch_access_token(access_token_url)

access_token = oauth_tokens["oauth_token"]
access_token_secret = oauth_tokens["oauth_token_secret"]
print('returned access_token', access_token)
print('returned access_token_secret', access_token_secret)
# Make the request
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=access_token_secret,
)

# Making the request
response = oauth.post(
    "https://api.twitter.com/2/tweets",
    json=payload,
)

if response.status_code != 201:
    raise Exception(
        "Request returned an error: {} {}".format(response.status_code, response.text)
    )

print("Response code: {}".format(response.status_code))

# Saving the response as JSON
json_response = response.json()
print(json.dumps(json_response, indent=4, sort_keys=True))

In [ ]:
from typing import Dict
import tweepy
from langchain.tools import BaseTool

CONSUMER_KEY = API_KEY
CONSUMER_SECRET = API_SECRET_KEY

class PostTweetTool(BaseTool):
    name:str = "Post tweet"
    description:str = "use this tool when you need to post a tweet"

    def _run(self, message:Dict) -> str:
        try:
            api = tweepy.Client(
                consumer_key=CONSUMER_KEY,
                consumer_secret=CONSUMER_SECRET,
                access_token=ACCESS_TOKEN,
                access_token_secret=ACCESS_TOKEN_SECRET,
            )
            tweet = api.create_tweet(text=message)
            print(tweet.data)
            return "Tweet posted successfully!"
        except Exception as e:
            return f"An error occurred posting tweet: {e}"
    def _arun(self):
        return "Not implemented"




test_input = {"message": "This is a revolutionary tweet! Wait to find out what it is!!!"}
post_tweet_tool = PostTweetTool()
result = post_tweet_tool.invoke(test_input)
print(result)

In [ ]:
from typing import Dict, Optional, Type
from pydantic import BaseModel, Field
import tweepy
from langchain.tools import BaseTool
from langchain_core.callbacks import (
    CallbackManagerForToolRun,
)
class AnswerTweetInput(BaseModel):
    tweet_id: str = Field(description="The tweet id to answer to")

class AnswerTweetTool(BaseTool):
    name: str = "Answer tweet"
    description: str = "use this tool when you need to answer to a tweet"
    args_schema: Type[BaseModel] = AnswerTweetInput
    def _run(
        self, tweet_id: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        try:

            api = tweepy.Client(
                consumer_key=CONSUMER_KEY,
                consumer_secret=CONSUMER_SECRET,
                access_token=ACCESS_TOKEN,
                access_token_secret=ACCESS_TOKEN_SECRET,
            )
            api.create_tweet(text='You are right!', in_reply_to_tweet_id=tweet_id)
            return "Answer tweet posted successfully!"
        except Exception as e:
            return f"An error occurred answering tweet: {e}"

    def _arun(self):
        return "Not implemented"


reply_tweet_tool = AnswerTweetTool()
result = reply_tweet_tool.invoke({"tweet_id": "1857078831418315145"})
print(result)

In [ ]:
# todo, add tool syntax

import tweepy
from langchain.tools import BaseTool

class ReadTweetsTool():
    # name:str = "Read tweets from timeline"
    # description:str = "use this tool when you need to read tweets from your timeline"

    def __init__(self):
        self.api = tweepy.Client(
            consumer_key=CONSUMER_KEY,
            consumer_secret=CONSUMER_SECRET,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET,
        )

    def _run(self) -> tweepy.Client.get_home_timeline:
        try:
            print(self.api.get_me())
            tweets = self.api.get_home_timeline()
            return tweets
        except Exception as e:
            return f"An error occurred while reading tweets: {e}"
    def _arun(self):
        return "Not implemented"


tool = ReadTweetsTool()
result = tool._run()
print(result)

In [ ]:
import tweepy
from typing import  Optional, Type
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain_core.callbacks import (
    CallbackManagerForToolRun,
)

class FollowUserInput(BaseModel):
    username: str = Field(description="The username of the user to follow")

class FollowUserTool(BaseTool):
    name: str = "Follow user"
    description: str = "useful for when you need to follow a user by their username"
    args_schema: Type[BaseModel] = FollowUserInput
    return_direct: bool = True



    def _run(
        self, username: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ):
        try:
            api = tweepy.Client(
                consumer_key=CONSUMER_KEY,
                consumer_secret=CONSUMER_SECRET,
                access_token=ACCESS_TOKEN,
                access_token_secret=ACCESS_TOKEN_SECRET,
                bearer_token=BEARER_TOKEN,
                )
            user = api.get_user(username=username)
            if not user.data:
                return f"Could not find user with username {username}"

            api.follow_user(user.data.id)
            return f"Successfully followed user {username}"
        except Exception as e:
            return f"An error occurred while following user: {str(e)}"

    async def _arun(self):
        return "Not implemented"

tool = FollowUserTool()
result = tool.invoke({"username": "brian_armstrong"})
print(result)

In [ ]:
# todo, add tool syntax
import tweepy
import json
from typing import List, Dict


class FollowUsersTool:
    def __init__(self):

        self.api = tweepy.Client(bearer_token=BEARER_TOKEN)
        self.oauth = tweepy.Client(
            consumer_key=CONSUMER_KEY,
            consumer_secret=CONSUMER_SECRET,
            access_token=ACCESS_TOKEN,
            access_token_secret=ACCESS_TOKEN_SECRET,
        )

    def follow_users(self, data: List[Dict]):

        try:
            # Extract handles from the JSON data
            handles = [account["handle"] for account in data]

            # Get users information using the handles
            users = self.api.get_users(usernames=handles).data
            print(f"Found users: {users}")

            user_ids = [user.id for user in users]

            for user_id in user_ids:
                self.oauth.follow_user(user_id)
                print(f"Followed user: {user_id}")

            return users
        except Exception as e:
            return f"An error occurred while getting users: {e}"


def follow_users_tool() -> Dict:
    with open("accounts.json", "r") as file:
        data = json.load(file)
    data = data[:100]
    follow_user = FollowUsersTool()
    result = follow_user.follow_users(data)
    return {"result": result}


result = follow_users_tool()
print(result)